# 🩻 Hyperparameter Tuning

In [1]:
import wandb
import torch
import ray
from ray import tune
from ray.air.integrations.wandb import WandbLoggerCallback
from ultralytics import YOLO
import os

In [2]:
# Initialize Ray with GPU support
if torch.cuda.is_available():
    ray.init(num_gpus=1, include_dashboard=False)
else:
    ray.init(include_dashboard=False)
    print("CUDA not available, using CPU")

2025-08-15 06:28:23,993	INFO worker.py:1927 -- Started a local Ray instance.


(train_yolo pid=66488) wandb: Currently logged in as: amswanson333 (synth-drone) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_yolo pid=66488) wandb: Tracking run with wandb version 0.21.0
(train_yolo pid=66488) wandb: Run data is saved locally in C:\Users\keyan\AppData\Local\Temp\ray\session_2025-08-15_06-28-21_989231_60456\artifacts\2025-08-15_06-30-56\train_yolo_2025-08-15_06-28-38\working_dirs\yolo_tuning_eda6d_00000\wandb\run-20250815_063100-y41qgvpi
(train_yolo pid=66488) wandb: Run `wandb offline` to turn off syncing.
(train_yolo pid=66488) wandb: Syncing run wobbly-pond-91
(train_yolo pid=66488) wandb:  View project at https://wandb.ai/synth-drone/yolo-tuning
(train_yolo pid=66488) wandb:  View run at https://wandb.ai/synth-drone/yolo-tuning/runs/y41qgvpi


(train_yolo pid=66488) Ultralytics 8.3.179  Python-3.12.11 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
(train_yolo pid=66488) engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.5690434417353024, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=0, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.9768476180619344, data=E:\Python\synthetic-data-for-object-detection\model\tuning.yaml, degrees=40.20942210656169, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.3501506992228135, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.8086629700564654, flipud=0.9245738808052208, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.4960804280991906, hsv_s=0.7455280890283875, hsv_v=0.15744822714630913, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003922413441904714, lrf=0.0014761624436959986, mask_ratio=4, ma

(train_yolo pid=62188) wandb: Currently logged in as: amswanson333 (synth-drone) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_yolo pid=62188) wandb: Tracking run with wandb version 0.21.0
(train_yolo pid=62188) wandb: Run data is saved locally in C:\Users\keyan\AppData\Local\Temp\ray\session_2025-08-15_06-28-21_989231_60456\artifacts\2025-08-15_06-30-56\train_yolo_2025-08-15_06-28-38\working_dirs\yolo_tuning_eda6d_00001\wandb\run-20250815_063100-mxco269v
(train_yolo pid=62188) wandb: Run `wandb offline` to turn off syncing.
(train_yolo pid=62188) wandb: Syncing run azure-lion-92
(train_yolo pid=62188) wandb:  View project at https://wandb.ai/synth-drone/yolo-tuning
(train_yolo pid=62188) wandb:  View run at https://wandb.ai/synth-drone/yolo-tuning/runs/mxco269v


(train_yolo pid=66488) AMP: checks passed 
(train_yolo pid=66488) train: Fast image access  (ping: 0.00.0 ms, read: 180.292.6 MB/s, size: 3742.8 KB)


...

(train_yolo pid=69680) Freezing layer 'model.23.dfl.conv.weight'
(train_yolo pid=69680) AMP: running Automatic Mixed Precision (AMP) checks...


(train_yolo pid=21292) wandb: Tracking run with wandb version 0.21.0
(train_yolo pid=21292) wandb: Run data is saved locally in C:\Users\keyan\AppData\Local\Temp\ray\session_2025-08-15_06-28-21_989231_60456\artifacts\2025-08-15_06-30-56\train_yolo_2025-08-15_06-28-38\working_dirs\yolo_tuning_eda6d_00007\wandb\run-20250815_063436-c1fgw13g
(train_yolo pid=21292) wandb: Run `wandb offline` to turn off syncing.
(train_yolo pid=21292) wandb: Syncing run mild-dawn-98
(train_yolo pid=21292) wandb:  View project at https://wandb.ai/synth-drone/yolo-tuning
(train_yolo pid=21292) wandb:  View run at https://wandb.ai/synth-drone/yolo-tuning/runs/c1fgw13g


(train_yolo pid=21292) Ultralytics 8.3.179  Python-3.12.11 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
(train_yolo pid=21292) engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.21201011350013965, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.20371535235608396, data=E:\Python\synthetic-data-for-object-detection\model\tuning.yaml, degrees=104.344993430274, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.4088161496984335, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5259930488053394, flipud=0.7878929924983153, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.30651142535623455, hsv_s=0.6237528235141581, hsv_v=0.7475269280760829, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00590886350122018, lrf=0.0015897056971674991, mask_ratio=4, 

(train_yolo pid=69680) train: Fast image access  (ping: 0.00.0 ms, read: 266.157.2 MB/s, size: 3742.8 KB)
(train_yolo pid=21292) AMP: checks passed 


train: Scanning E:\Python\synthetic-data-for-object-detection\data\processed\train.cache... 84995 images, 6841 backgrounds, 0 corrupt: 100%|██████████| 84995/84995 [00:00<?, ?it/s]
(train_yolo pid=21292) Traceback (most recent call last):
(train_yolo pid=21292)   File "<string>", line 1, in <module>
(train_yolo pid=21292)   File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\multiprocessing\spawn.py", line 122, in spawn_main
(train_yolo pid=21292)     exitcode = _main(fd, parent_sentinel)
(train_yolo pid=21292)                ^^^^^^^^^^^^^^^^^^^^^^^^^^
(train_yolo pid=21292)   File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\multiprocessing\spawn.py", line 132, in _main
(train_yolo pid=21292)     self = reduction.pickle.load(from_parent)
(train_yolo pid=21292)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(train_yolo pid=21292)   File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\site-packages\torch\__init__.py", line 278, in <module>
(train_yolo pid=21292)     _load_dll_libraries()
(train_

In [3]:
wandb.login(key='f48c90599b9faeaaa6bcfe64bea8f132a8e3bc41')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\keyan\_netrc
wandb: Currently logged in as: amswanson333 (synth-drone) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
# YOLO Model YAML file

model_path = 'model'

yaml_filename = 'tuning.yaml'

yaml_content = """
path: data/processed/  # dataset root dir (leave empty for HUB)
train: train  # train images (relative to 'path')
val:   val    # val images (relative to 'path')
test:  test   # test images (optional)

names:
  0: drone
"""

In [5]:
# # Write the YAML config file

# with open(os.path.join(model_path, yaml_filename), 'w') as f:
#     f.write(yaml_content)

In [ ]:
# Load pre-trained YOLO 11 model
model = YOLO('yolo11x.pt')

In [ ]:
config = {
    'batch_size': 16,
    'learning_rate': 0.001,
    'weight_decay': 0.0005,
    'image_size': 640
}

In [ ]:
results = model.train(
    data='E:\\Python\\synthetic-data-for-object-detection\\model\\tuning.yaml',
    epochs=5,
    device=0,
    **config
)

In [7]:
def train_yolo(config):
    # Initialize W&B
    wandb.init(project="yolo-tuning", config=config)

    # Load pre-trained YOLO model
    model = YOLO('yolo11x.pt')

    # Train the model with the hyperparameters from the config
    results = model.train(
        data='E:\\Python\\synthetic-data-for-object-detection\\model\\tuning.yaml',
        epochs=50,
        patience=15,
        device=0,
        **config
    )

    # Report metrics to Tune and Log to W&B
    metrics = {
        "mAP": results.results_dict.get('metrics/mAP50-95(B)', 0),
        "precision": results.results_dict.get('metrics/precision(B)', 0),
        "recall": results.results_dict.get('metrics/recall(B)', 0),
        "fitness": results.results_dict.get('fitness', 0)
    }
    tune.report(**metrics)
    wandb.log(metrics)

    # Finish W&B run
    wandb.finish()

In [8]:
search_space = {
    "batch": tune.choice([16, 32, 64]),
    "imgsz": tune.choice([640, 1280]),
    "optimizer": tune.choice(['SGD', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSprop']),
    "multi_scale": tune.choice([True, False]),
    "cos_lr": tune.choice([True, False]),
    "close_mosaic": tune.choice([0, 5, 10, 15]),
    "lr0": tune.loguniform(1e-4, 1e-2),
    "lrf": tune.loguniform(1e-4, 1e-2),
    "momentum": tune.uniform(0.8, 0.95),
    "weight_decay": tune.loguniform(1e-4, 1e-2),
    "dropout": tune.uniform(0, 0.5),
    
    "hsv_h": tune.uniform(0, 1.0),
    "hsv_s": tune.uniform(0, 1.0),
    "hsv_v": tune.uniform(0, 1.0),
    "degrees": tune.uniform(0, 180.0),
    "translate": tune.uniform(0, 1.0),
    "scale": tune.uniform(0.5, 1.0),
    "shear": tune.uniform(-180, 180),
    "perspective": tune.uniform(0, 0.001),
    "flipud": tune.uniform(0, 1.0),
    "fliplr": tune.uniform(0, 1.0),
    "bgr": tune.uniform(0, 1.0),
    "mosaic": tune.uniform(0, 1.0),
    "mixup": tune.uniform(0, 1.0),
    "cutmix": tune.uniform(0, 1.0)
}

In [9]:
# Run the hyperparameter tuning
# search_alg = BayesOptSearch(metric="mAP", mode="max")

tune_config = tune.TuneConfig(
    metric="mAP",
    mode="max",
    num_samples=10,
    max_concurrent_trials=2,
    trial_dirname_creator=lambda trial: f"yolo_tuning/{trial.trial_id}"
)
run_config = tune.RunConfig(
    name="yolo_tuning",
    storage_path=os.path.join(model_path, 'ray_results'),
    callbacks=[WandbLoggerCallback(project="yolo-tuning", log_config=True)]
)

In [10]:
tuner = tune.Tuner(
    trainable=train_yolo,
    param_space=search_space,
    tune_config=tune_config
)

In [11]:
torch.cuda.device_count()

1

In [ ]:
analysis = tuner.fit()

2025-08-15 06:32:03,991	ERROR tune_controller.py:1331 -- Trial task failed for trial train_yolo_eda6d_00001
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\site-packages\ray\air\execution\_internal\event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\site-packages\ray\_private\auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\site-packages\ray\_private\client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\ray_yolo\Lib\site-packages\ray\_private\worker.py", line 2858, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "

: 